In [17]:
import sys
import os

# Get the current notebook's directory
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
# Get the project root (two levels up from the notebook)
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
# Add the project root to Python path
sys.path.append(project_root)
print(f"Project root: {project_root}")

# Questrade Market Data Management System Notebook

This notebook demonstrates how to use the market data management system for Questrade API. The system fetches, stores, and analyzes candlestick data from Questrade's API using SQLite or PostgreSQL databases, with support for Parquet file conversion.

## 1. Setup and Configuration


In [18]:
import sys
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sqlite3
import time

# Configure the data management system
import src.local_symbols as local_symbols
import src.qt_api.qt_api as qt
import src.local_candles as local_candles

# Set up configuration variables
# Change these variables in src/local_candles.py 
DB_SYSTEM = 'sqlite'  # Could be 'postgres' for production 
DEBUG = True

# Set plot style
plt.style.use('ggplot')
sns.set_theme(style="darkgrid")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 2. Database Initialization

Let's first initialize our database and make sure the required tables exist.


In [ ]:
# First, ensure our data directory exists
local_candles.ensure_data_directory()

# Initialize the database
local_candles.initialize_db()

# Apply database optimizations (indexes)
local_candles.optimize_database()

# Print database location
data_dir = Path('../data')
db_path = data_dir / 'candles.db'
print(f"Database initialized at: {db_path}")

# Check database tables
with local_candles.get_db_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("\nDatabase tables:")
    for table in tables:
        print(f"- {table[0]}")
DEBUG = False

## 3. Retrieving and Storing Candle Data

Now, let's explore how to fetch and store candle data for specific symbols.


In [20]:
# Define a symbol we want to work with
test_symbol = "AAPL"  # Apple Inc.


In [ ]:
# First, let's see how to fetch candle data directly from Questrade API
print(f"Fetching candle data for {test_symbol} from Questrade API...")
candle_df = local_candles.get_candle_df(test_symbol, days_back=7, interval="OneMinute")
print(f"Retrieved {len(candle_df)} candles")

# Display a sample of the data
display(candle_df.head())

In [ ]:
# Now, let's update the database with this data
print(f"\nUpdating database with candle data for {test_symbol}...")
local_candles.update_candles_for_symbol(test_symbol, interval="OneMinute")
print("Update complete")

In [ ]:
# Retrieve the data from the database to verify it was stored
candles_from_db = local_candles.get_candle_from_db(test_symbol, interval="OneMinute")
print(f"Retrieved {len(candles_from_db)} candles from database")

# Display a sample of the data from the database
display(candles_from_db.head())

## 4. Working with Time Series Data

Let's explore how to work with the time series data we've collected.


In [ ]:
# Get data for a specific time period
start_date = (datetime.datetime.now() - datetime.timedelta(days=5)).strftime("%Y-%m-%d")
end_date = datetime.datetime.now().strftime("%Y-%m-%d")

period_candles = local_candles.get_candles_for_period(
    test_symbol, 
    start_date, 
    end_date, 
    interval="OneMinute"
)

print(f"Retrieved {len(period_candles)} candles for period {start_date} to {end_date}")

# Convert the 'start' column to datetime for proper plotting
period_candles['start'] = pd.to_datetime(period_candles['start'])

# Create a simple plot of closing prices
plt.figure(figsize=(12, 6))
plt.plot(period_candles['start'], period_candles['close'])
plt.title(f"{test_symbol} Closing Prices")
plt.xlabel('Time')
plt.ylabel('Price ($)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Get the latest price
latest_price = local_candles.get_latest_price(test_symbol)
print(f"Latest price for {test_symbol}: ${latest_price:.2f}")


## 5. Working with Multiple Symbols

Let's update multiple symbols and analyze a small portfolio.


In [ ]:
# Define a list of symbols for our example portfolio
portfolio_symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

# Let's manually update our symbols one by one
for symbol in portfolio_symbols:
    try:
        local_candles.update_candles_for_symbol(symbol, interval="OneMinute")
        print(f"Updated {symbol} successfully")
    except Exception as e:
        print(f"Error updating {symbol}: {e}")

In [ ]:
# Alternatively, we could use update_all_symbols() to update all symbols in the database
UPDATE_ALL_SYMBOLS = True # Change to True to update all symbols
if UPDATE_ALL_SYMBOLS:
    print("Updating all symbols in database...")
    local_candles.update_all_symbols(backup_db=True)


In [ ]:



# Now let's fetch the latest prices for our portfolio
portfolio_prices = {}
for symbol in portfolio_symbols:
    price = local_candles.get_latest_price(symbol)
    portfolio_prices[symbol] = price

# Create a DataFrame to display the portfolio
portfolio_df = pd.DataFrame({
    'Symbol': portfolio_prices.keys(),
    'Latest Price': portfolio_prices.values()
})

# Display the portfolio
display(portfolio_df)

# Create a bar chart of the portfolio prices
plt.figure(figsize=(10, 6))
plt.bar(portfolio_df['Symbol'], portfolio_df['Latest Price'], color='skyblue')
plt.title('Portfolio Latest Prices')
plt.xlabel('Symbol')
plt.ylabel('Price ($)')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


## 6. Converting Database to Parquet Format

Parquet is a columnar storage format that provides efficient data compression and encoding schemes, making it good for time series data.



In [ ]:
run_parquet = True # Change to True to run the conversion
if run_parquet:
    # Convert the database to Parquet format
    print("Converting database to Parquet format...")
    local_candles.db_to_parquet()

# Try to load the Parquet file
try:
    # Specify the file_name parameter required by load_parquet function
    file_name = "all_candles"  # Use the base name without timestamp or extension
    parquet_df = local_candles.load_parquet(file_name)
    print(f"Loaded Parquet file with {len(parquet_df)} records")
    
    # Display the first few rows
    display(parquet_df.head())
    
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error loading parquet file: {e}")

## 7. Visualizing Candle Data

Let's use our custom function to plot OHLC data for a specific symbol.



In [ ]:
# Let's use our plotting function to visualize a stock
try:
    symbol_to_plot = "AAPL"
    
    # Check if the symbol exists in the parquet data
    if symbol_to_plot in parquet_df.index.get_level_values('item_id'):  # symbol is in the item_id index
        print(f"Plotting {symbol_to_plot} from Parquet data...")
        local_candles.parquet_plot_ohlc(parquet_df, symbol_to_plot, interval='OneMinute', n_days=120)
    else:
        print(f"Symbol '{symbol_to_plot}' not found in Parquet data, trying database...")
        
        # Option 2: Plot from database
        candles = local_candles.get_candle_from_db(symbol_to_plot, interval="OneMinute")
        
        if not candles.empty:
            # Convert to the format expected by the plotting function
            # Use utc=True to avoid FutureWarning about mixed time zones
            candles['start'] = pd.to_datetime(candles['start'], utc=True)
            # Set the multi-index with item_id (symbol) and timestamp (start)
            candles.rename(columns={'symbol': 'item_id'}, inplace=True)
            candles.set_index(['item_id', 'start'], inplace=True)
            candles.index.names = ['item_id', 'timestamp']
            
            # Plot the data
            local_candles.parquet_plot_ohlc(candles, symbol_to_plot, interval='OneMinute', n_days=1)
        else:
            print(f"No data found for symbol '{symbol_to_plot}' with interval 'OneMinute'")
        
except Exception as e:
    print(f"Error plotting data: {e}")


## 8. Preparing Data for AutoGluon (Time Series Forecasting)

The system includes functionality to prepare data for time series forecasting with AutoGluon.


In [ ]:
# This would update all symbols and prepare a combined file for AutoGluon
# Note: This can take a long time if you have many symbols
print("To prepare data for AutoGluon, you would run:")
print("ag_data = local_candles.prepare_parquet_autogluon(update_db=True)")
print("\nThis function:")
print("1. Updates candle data for all symbols")
print("2. Formats the data for time series forecasting")
print("3. Saves as a Parquet file with timestamp")
print("4. Creates a backup of the database")

# Instead of running the full process, let's demonstrate with a small subset
print("\nDemonstrating with a small subset (no updates)...")

# You could run this once a day to update the candle database and get a fresh parquet file
PREPARE_AUTOGLUON = True # Change to True to run the conversion
if PREPARE_AUTOGLUON:
    local_candles.prepare_parquet_autogluon(update_db=False) # Set update_db to True if needed


## 9. Database Maintenance and Backup

In [ ]:
# Create a backup of the database
print("Creating database backup...")
local_candles.backup_database()

# Verify the backup directory
backup_dir = Path('../data/backups')
if backup_dir.exists():
    backups = list(backup_dir.glob("*.db"))
    print(f"Found {len(backups)} database backups:")
    for backup in backups:
        backup_size = backup.stat().st_size / (1024 * 1024)  # Convert to MB
        print(f"- {backup.name} ({backup_size:.2f} MB)")
else:
    print("No backup directory found")

## 10. System Performance Analysis

Let's analyze how the system is performing in terms of data storage efficiency.


In [ ]:
# Check database size
db_size_bytes = db_path.stat().st_size
db_size_mb = db_size_bytes / (1024 * 1024)
print(f"Database size: {db_size_mb:.2f} MB")

# Count the number of candles in the database
with local_candles.get_db_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM candles")
    candle_count = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(DISTINCT symbol) FROM candles")
    symbol_count = cursor.fetchone()[0]
    
    cursor.execute("SELECT symbol, COUNT(*) as candle_count FROM candles GROUP BY symbol ORDER BY candle_count DESC LIMIT 10")
    top_symbols = cursor.fetchall()

print(f"Total candles in database: {candle_count:,}")
print(f"Number of unique symbols: {symbol_count}")
print(f"Average candles per symbol: {candle_count/symbol_count if symbol_count else 0:,.0f}")

# Print top symbols by candle count
print("\nTop symbols by candle count:")
for symbol, count in top_symbols:
    print(f"- {symbol}: {count:,} candles")

# Calculate efficiency metrics
bytes_per_candle = db_size_bytes / candle_count if candle_count else 0
print(f"\nStorage efficiency: {bytes_per_candle:.2f} bytes per candle")

# Check database growth potential
estimated_daily_candles_per_symbol = 390  # ~6.5 hours × 60 minutes for US market
estimated_daily_growth = estimated_daily_candles_per_symbol * symbol_count * bytes_per_candle / (1024 * 1024)
print(f"Estimated daily database growth: {estimated_daily_growth:.2f} MB")
print(f"Estimated annual database growth: {estimated_daily_growth * 252:.2f} MB (252 trading days)")


## 11. Conclusion

This notebook has demonstrated the key functionality of the Questrade market data management system. The system provides:

1. Database initialization and optimization
2. Efficient storage of candle data from Questrade API
3. Flexible query capabilities for time series analysis
4. Support for both SQL and Parquet storage formats
5. Visualization tools for market data
6. Data preparation for advanced time series forecasting

The SQL-based approach offers several advantages:
- Efficient storage with indexes for fast queries
- Support for complex time-based filtering
- Easy backup and maintenance
- Integration with both SQLite (for development) and PostgreSQL (for production)

With this system, you can build a robust financial data pipeline for algorithmic trading, portfolio analysis, or market research applications.